<img style="float: left; margin: 30px 15px 15px 15px;" src="https://oci02.img.iteso.mx/Identidades-De-Instancia/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg" width="400" height="600" /> 
    
    
##  Teoría Pos-Moderna de Portafolios: Práctica
    
### Portafolios de Inversión

    Mtro. Sean Nicolás González Vázquez

---

###  0.- Introducción
    
En la sesión anterior, exploramos varios métodos de la **Teoría Posmoderna de Portafolios (TPMP)**, que se enfoca en la optimización de portafolios separando claramente el riesgo en dos componentes: el downside y el upside risk. 
    
A grandes rasgos, esta teoría amplía la clásica Teoría Moderna de Portafolios, al poner énfasis en los riesgos que afectan negativamente las inversiones, y aquellos que representan oportunidades de ganancia.

La principal contribución de este enfoque radica en su capacidad para descomponer el riesgo en dos categorías clave:

+ **Downside risk**: Riesgo de incurrir en pérdidas, es decir, cuando los rendimientos caen por debajo de un umbral establecido, como podría ser un benchmark o simplemente rendimientos negativos.   
    
    
+ **Upside risk**: Riesgo (potencial) de obtener ganancias.   
    
    
+ Mientras que el downside implica posibles pérdidas para el portafolio, el upside representa las oportunidades de obtener rendimientos positivos.

Dentro de esta estructura, profundizamos en **dos métodos de Asset Allocation** basados en esta separación de riesgos: el método de la **Mínima Semivarianza** y el método de **Máximo Omega**. 
    
Ambos enfoques proporcionan alternativas interesantes a los modelos tradicionales propuestos por Markowitz y Sharpe.   
    
El primero, la Mínima Semivarianza, es una extensión del clásico método de Mínima Varianza, ya que se centra exclusivamente en minimizar el riesgo a la baja (downside risk), ignorando las fluctuaciones al alza. El segundo, el Máximo Omega, es una alternativa al Ratio de Sharpe, con la diferencia clave de que se enfoca en maximizar la relación entre las ganancias potenciales (upside) y las pérdidas posibles (downside), proporcionando una evaluación más matizada del rendimiento ajustado al riesgo.  

> Con este contexto, implementaremos ambos métodos, utilizando optimización montecarlo.

---

### 1.- Descarga de Datos y Obtención de Métricas

En este ejercicio, vamos a construir un portafolio compuesto por siete activos financieros previamente seleccionados:

+ PG: Procter & Gamble
+ COST: Costco Wholesale
+ KO: Coca-Cola
+ WMT: Walmart
+ CLX: Clorox
+ K: Kellogg's
+ KHC: Kraft Heinz

Como puedes observar, todos estos activos pertenecen a compañías que producen bienes de consumo básico, tales como productos de limpieza, alimentos y bebidas, entre otros. 


Estos activos forman parte de un sector específico conocido como Consumer Staples. Este sector es considerado anticíclico por naturaleza. Por ello, se le clasifica como un sector defensivo, ideal para inversiónistas que buscan estabilidad y son aversos al riesgo.

> **Consumer Staples:** Hace referencia a bienes de consumo necesarios para la vida diaria, como alimentos, bebidas, productos de higiene personal y del hogar. Estos productos suelen ser demandados de forma constante, independientemente del ciclo económico.  

> **Sector Defensivo:** Se refiere a aquellos sectores de la economía cuyos activos tienden a ser menos volátiles y más resilientes en tiempos de crisis o desaceleraciones económicas. Las empresas en estos sectores producen bienes y servicios esenciales, lo que les permite mantener una demanda relativamente estable incluso en condiciones adversas del mercado.


Como gestor del portafolio (portfolio manager), tu objetivo es encontrar las ponderaciones óptimas que minimicen la semivarianza del portafolio y maximicen el ratio de Omega.

In [27]:
# Importación de Librerías
import numpy as np
import pandas as pd
import yfinance as yf

In [28]:
# Descarga de Datos
prices=yf.download(['PG', 'COST', 'KO', 'WMT', 'CLX', 'K', 'KHC'], 
                    start='2020-01-01', end='2025-06-17')['Close']

[*********************100%***********************]  7 of 7 completed


In [29]:
# Calculo de rendimientos
rets = prices.pct_change().dropna()

In [30]:
# Calcular rendimiento promedio
ret_mean = rets.mean()
ret_mean

Ticker
CLX     0.000109
COST    0.001070
K       0.000412
KHC     0.000182
KO      0.000391
PG      0.000380
WMT     0.000798
dtype: float64

In [31]:
# Calcular matriz de correlación
corr = rets.corr()
corr

Ticker,CLX,COST,K,KHC,KO,PG,WMT
Ticker,,,,,,,
CLX,1.000000,0.336275,0.411293,0.368383,0.326680,0.504207,0.324792
COST,0.336275,1.000000,0.248196,0.338685,0.439561,0.502138,0.620015
K,0.411293,0.248196,1.000000,0.533076,0.450513,0.520562,0.317309
KHC,0.368383,0.338685,0.533076,1.000000,0.575052,0.557508,0.359823
KO,0.326680,0.439561,0.450513,0.575052,1.000000,0.655225,0.389985
PG,0.504207,0.502138,0.520562,0.557508,0.655225,1.000000,0.495011
WMT,0.324792,0.620015,0.317309,0.359823,0.389985,0.495011,1.000000


---

### 2.- Portafolio Eficiente en Mínima Semivarianza  
    
  
###   **Mínima Semivarianza**

Método de la optimización de portafolios que busca minimizar el potencial riesgo de pérdida (*downside risk*) de un portafolio de inversión. ´


**Separar los rendimientos por debajo de 0 (pérdidas), las ganancias convertirlas en cero.**

$$R_{below, i} = min(r_i, 0)$$


In [32]:
R_below_zero = rets[rets < 0].fillna(0)
R_below_zero

Ticker,CLX,COST,K,KHC,KO,PG,WMT
Date,,,,,,,
2020-01-03,0.000000,0.000000,0.000000,-0.011705,-0.005456,-0.006726,-0.008828
2020-01-06,0.000000,0.000000,-0.000882,0.000000,-0.000366,0.000000,-0.002036
2020-01-07,-0.012062,-0.001576,0.000000,-0.017566,-0.007683,-0.006192,-0.009264
2020-01-08,0.000000,0.000000,0.000000,-0.005527,0.000000,0.000000,-0.003432
2020-01-09,0.000000,0.000000,0.000000,-0.000327,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
2025-06-10,-0.004784,0.000000,-0.006830,0.000000,0.000000,0.000000,-0.001334
2025-06-11,-0.004965,-0.010414,0.000000,-0.006393,-0.003870,-0.004483,-0.015619
2025-06-12,0.000000,0.000000,-0.017032,0.000000,0.000000,0.000000,-0.010125


**Calcular downside risk para cada activo.**

$$\sigma_{d, i} = \sigma(R_{below, i})$$

In [33]:
downside_risk = R_below_zero.std()
downside_risk

Ticker
CLX     0.010789
COST    0.009327
K       0.008579
KHC     0.010611
KO      0.008446
PG      0.008361
WMT     0.008898
dtype: float64

**Calcular matriz de Semivarianza.**

Antes de encontrar la matriz de semivarianza, simplifiquemos el cálculo......

Recordando, la varianza de dos activos financieros si tenemos la correlación entre ambos y sus desviaciones estandár individuales esta dada por $\sigma^2_{i, j} = \rho_{i, j} \sigma_i \sigma_j$. Para calcular la semi-varianza entre dos activos financieros utilizamos la misma fórmula, simplemente cambiando la desviación estandár por la semi-desviación (el *downside risk*), es decir, la semivarianza para dos activos financieros esta dada por $\sigma^2_{d(i, j)} = \rho_{i, j} \sigma_{(d, i)} \sigma_{(d, j)}$.

Esto debe hacerse iterativamente hasta construir la matriz de semivarianza, como vimos en la clase pasada. 

Para simplificar el ejercicio, podemos tomar el vector de $nx1$ de los downside risk individuales:

$$\sigma_d = \begin{bmatrix}
\sigma_{d, 1} \\
\sigma_{d, 2} \\
\vdots \\
\sigma_{d, n}
\end{bmatrix}$$

Y multiplicarla matricialmente por su transpuesta:

$$\sigma_d \sigma_d^T = 
\begin{bmatrix}
\sigma_{d, 1} \\
\sigma_{d, 2} \\
\vdots \\
\sigma_{d, n}
\end{bmatrix}
\begin{bmatrix}
\sigma_{d, 1} & \sigma_{d, 2} & \cdots & \sigma_{d, n}
\end{bmatrix}
=
\begin{bmatrix}
\sigma_{d, 1}^2 & \sigma_{d, 1} \sigma_{d, 2} & \cdots & \sigma_{d, 1} \sigma_{d, n} \\
\sigma_{d, 2} \sigma_{d, 1} & \sigma_{d, 2}^2 & \cdots & \sigma_{d, 2} \sigma_{d, n} \\
\vdots & \vdots & \ddots & \vdots \\
\sigma_{d, n} \sigma_{d, 1} & \sigma_{d, n} \sigma_{d, 2} & \cdots & \sigma_{d, n}^2
\end{bmatrix}$$

Notesé que esto equivale a multiplicar el elemento $i$ con el $j$ de la matriz de semivarianza, donde la diagonal son los downside al cuadrado (puedes utilizar la función de `np.multiply`). Finalmente realizamos el producto Hadamard de esta matriz con la matriz de correlación y con esto obtenemos la matriz de semivarianza.

$$S = \begin{bmatrix}
1 & \rho_{12} & \cdots & \rho_{1n} \\
\rho_{21} & 1 & \cdots & \rho_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
\rho_{n1} & \rho_{n2} & \cdots & 1
\end{bmatrix} \circ \begin{bmatrix}
\sigma_{d, 1}^2 & \sigma_{d, 1} \sigma_{d, 2} & \cdots & \sigma_{d, 1} \sigma_{d, n} \\
\sigma_{d, 2} \sigma_{d, 1} & \sigma_{d, 2}^2 & \cdots & \sigma_{d, 2} \sigma_{d, n} \\
\vdots & \vdots & \ddots & \vdots \\
\sigma_{d, n} \sigma_{d, 1} & \sigma_{d, n} \sigma_{d, 2} & \cdots & \sigma_{d, n}^2
\end{bmatrix}$$

El producto [Hadamard](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)#:~:text=In%20mathematics,%20the%20Hadamard%20product%20also%20known%20as%20the) es una multiplicación elemento a elemento de cada matriz según su posición específica (para realizarla en python, puedes utilizar simplimente `*`). Al realizar esta operación obtenemos la matriz de semivarianza para el portafolio:

$$S=
\begin{bmatrix}
\sigma_{d, 1}^2 & \rho_{12} \sigma_{d, 1} \sigma_{d, 2} & \cdots & \rho_{1n} \sigma_{d, 1} \sigma_{d, n} \\
\rho_{21} \sigma_{d, 2} \sigma_{d, 1} & \sigma_{d, 2}^2 & \cdots & \rho_{2n} \sigma_{d, 2} \sigma_{d, n} \\
\vdots & \vdots & \ddots & \vdots \\
\rho_{n1} \sigma_{d, n} \sigma_{d, 1} & \rho_{n2} \sigma_{d, n} \sigma_{d, 2} & \cdots & \sigma_{d, n}^2
\end{bmatrix}$$


**Calcular semivarianza del portafolio.**

Con la matriz de semivarianza, podemos obtener la semivarianza de un portafolio de inversión con la fórmula:

$$s = w^T S w$$

##### Convertir a Vector 

In [34]:
downside_risk = np.array(downside_risk)
downside_risk

array([0.01078914, 0.00932664, 0.00857894, 0.01061099, 0.00844643,
       0.00836121, 0.00889807])

In [35]:
semivar_matrix = downside_risk.reshape(len(rets.keys()),1) @ downside_risk.reshape(1,len(rets.keys())) * corr
semivar_matrix

Ticker,CLX,COST,K,KHC,KO,PG,WMT
Ticker,,,,,,,
CLX,0.000116,0.000034,0.000038,0.000042,0.000030,0.000045,0.000031
COST,0.000034,0.000087,0.000020,0.000034,0.000035,0.000039,0.000051
K,0.000038,0.000020,0.000074,0.000049,0.000033,0.000037,0.000024
KHC,0.000042,0.000034,0.000049,0.000113,0.000052,0.000049,0.000034
KO,0.000030,0.000035,0.000033,0.000052,0.000071,0.000046,0.000029
PG,0.000045,0.000039,0.000037,0.000049,0.000046,0.000070,0.000037
WMT,0.000031,0.000051,0.000024,0.000034,0.000029,0.000037,0.000079


**Minimizar semivarianza cambiando ponderaciones.**

$$min_w \hspace{0.5cm} \sigma_d^2 = w^T S w$$
    
$$s.a. \hspace{0.5cm} \sum_{i=1}^n w_i = 1$$
 
$$\hspace{0.8cm} w_i > 0 $$



In [36]:
# Simular portafolios con montecarlo
n_assets = len(rets.keys())
n_assets
# Numero de simulaciones
n_simulacions = 10000
# Generar pesos 
w = np.random.dirichlet(np.ones(n_assets), n_simulacions)


In [37]:
# Obtener la mínima semivarianza
semivar_list=[w.T @ semivar_matrix @ w for w in w ]
semivar_list 

[4.984579924214296e-05,
 4.7341929337042623e-05,
 5.061712708031853e-05,
 5.2839841259332575e-05,
 5.1662155395103904e-05,
 5.093294752830167e-05,
 5.8759578834538566e-05,
 5.537003293514981e-05,
 6.414751604451519e-05,
 4.769669507191457e-05,
 5.1364573239238946e-05,
 4.962419290850812e-05,
 7.101710697263294e-05,
 4.916111350127261e-05,
 4.649191556777486e-05,
 4.7214352506213386e-05,
 4.8581939823813636e-05,
 5.655351773156283e-05,
 4.885437162695978e-05,
 4.6826748434709426e-05,
 4.812324827011549e-05,
 4.573350577365126e-05,
 4.555931170806194e-05,
 4.464276218786494e-05,
 6.565267206200281e-05,
 5.032260818334214e-05,
 4.940518632779211e-05,
 5.2454635273541335e-05,
 4.9365948091970233e-05,
 4.965018537447217e-05,
 4.6433835631248354e-05,
 5.948101362387232e-05,
 4.649579575829401e-05,
 5.505454810094953e-05,
 5.103020889222402e-05,
 6.607716579008005e-05,
 4.5309945227543375e-05,
 4.632635998953262e-05,
 5.728225764815147e-05,
 4.679386219535389e-05,
 5.583995596616728e-05,
 4.3

In [38]:
dict(zip(rets.keys(), w[np.argmin(semivar_list)]))

{'CLX': 0.10068543074961385,
 'COST': 0.16290281058274989,
 'K': 0.2968486688635524,
 'KHC': 0.0017824829326532835,
 'KO': 0.2454747902488431,
 'PG': 0.010654379003868317,
 'WMT': 0.18165143761871919}

#### ¿Qué pasa con las métricas del portafolio ?

---

###  3.- Portafolio Eficiente en Máximo Omega
    
    
Portafolio que maximiza el ratio omega de un portafolio de inversión, el ratio Omega mide la relación entre el upside risk (potencial de ganancias) y el downside risk (potencial de pérdidas) de un activo financiero.

**Separar los rendimientos en dos, aquellos por debajo de cero y los que están por encima de cero.**

$$R_{below, i} = min(r_i, 0)$$

$$R_{beneath, i} = max(r_i, 0)$$

**Calcular downside y upside risk para cada activo.**

$$\sigma_{d, i} = \sigma(R_{below, i})$$

$$\sigma_{u, i} = \sigma(R_{beneath, i})$$

**Calcular Omega individual para cada activo.**

$$\Omega_i = \frac{\sigma_{u, i}}{\sigma_{d, i}}$$


**Calcular Omega del portafolio y maximizarla cambiando ponderaciones.**

$$max_w \hspace{0.5cm} \Omega_p = \sum_{i=1}^{n} \Omega_i * w_i$$
    
$$s.a. \hspace{0.5cm} \sum_{i=1}^n w_i = 1$$
 
$$\hspace{0.8cm} w_i > 0 $$

In [39]:
# Simular portafolios con montecarlo





In [40]:
# Obtener el maximo omega


In [41]:
# Obtener los pesos del portafolio con maximo omega




---

### 4.- Target Minimum Semivariance Portfolio
    

Ahora, supón que deseas optimizar las ponderaciones del portafolio considerando aquellos activos que tienen mayor probabilidad de superar a tu *benchmark*. Para ello, emplearás una estrategia basada en la *target semivariance*.

En este caso, asume que tu *benchmark* es el ETF `KXI`, que sigue el sector de bienes de consumo global. 
    
El objetivo es encontrar las ponderaciones óptimas que minimicen el riesgo de no superar el rendimiento de `KXI`. 
    
    
Recuerda que este enfoque ajustado al *benchmark* permite identificar aquellos activos que tienen un mejor desempeño relativo y, por lo tanto, una mayor probabilidad de generar *alpha*. Al minimizar la semivarianza relativa al *benchmark*, estarás optimizando el portafolio para que minimice el riesgo de rendimientos inferiores al de `KXI`.

Recuerda que los pasos son reiterativos y siguen la misma estructura que el método de mínima semivarianza, solo que ahora restarás los rendimientos al rendimiento del *benchmark* a los de tus activos.

In [42]:
# Descarga de Precios del Benchmark


In [43]:
# Obtener rendimientos del benchmark




In [44]:
# Sustraer rendimiento del benchmark de rendimientos individuales




In [45]:
# Separar los rendimientos por debajo de cero




In [46]:
# Calcular target downside risk




In [47]:
# Calcular la matriz de target semivariance




In [48]:
# Simular portafolios con montecarlo




In [49]:
# Obtener la mínima target semivariance


In [50]:
# Obtener los pesos del portafolio con mínima semivarianza target




    ¡Excelente! Ahora comparemos las ponderaciones obtenidas en el método de semivarianza normal con target semivariance, responde: ¿Hay diferencias? ¿Como las interpretas?

---

### 5.- Reflexión

> **El método de la semivarianza y el ratio omega** son dos alternativas interesantes para la optimización de portafolios de inversión, ya que **replantean el concepto tradicional del riesgo financiero**.

> **¿Cuál es mejor?** No hay una respuesta definitiva; **la elección** del método "adecuado" **depende de varios factores**, como el perfil del inversionista, el comportamiento histórico de los activos seleccionados (que puede hacer que un método funcione mejor que otro) y las restricciones u objetivos específicos del gestor.

> Es importante resaltar nuevamente que estos **métodos solo son tan efectivos como lo sean los activos incluidos en el portafolio**.

> Hasta ahora, no hemos comparado las distintas estrategias de asignación de activos, sino que hemos enfocado en cómo determinar los pesos óptimos. **En las próximas sesiones, nos centraremos en la comparación y selección de estrategias de inversión**. Esto te permitirá, con un conjunto de activos que conforman un portafolio, evaluar sus características y elegir la mejor estrategia basándote en las métricas de los múltiples portafolios óptimos.

---

###  Tarea 5

En equipos de dos integrantes, respondan los siguientes puntos y entreguen un archivo `html` con sus resultados. No olviden incluir los nombres de los integrantes en el archivo:

**Punto a)** Implementen tres funciones que optimicen portafolios utilizando: mínima semivarianza, ratio omega y semivarianza objetivo, empleando la función `scipy.optimize.minimize`.

**Punto b)** Construyan un portafolio compuesto por 5 activos financieros y optimícenlo con los tres métodos. Utilicen las funciones desarrolladas en el punto a) para realizar la optimización. Recuerden que, para el portafolio con semivarianza objetivo, deberán seleccionar un benchmark adecuado y consistente con los activos elegidos. Como respuesta a este punto, se esperan las ponderaciones eficientes y el valor óptimo de la función objetivo para cada método de Asset Allocation.

**Punto c)** Escriban una breve conclusión sobre las ventajas y desventajas que observan entre los métodos TMP y TPMP.

**Extra:** El equipo que logre la menor semivarianza global en su portafolio optimizado por mínima semivarianza recibirá 10 puntos extra para el segundo examen parcial. La selección de activos con una buena relación entre upside y downside será clave para obtener estos puntos.

